In [ ]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt

from skimage.transform import rescale, resize
from skimage.io import imread, imsave
from skimage.filters import meijering, gaussian, threshold_otsu
from skimage.morphology import opening, closing, disk, remove_small_objects
from skimage.morphology import convex_hull_image, binary_closing
from skimage.measure import label, regionprops
from scipy import ndimage as ndi

import math

from glob import glob
import os

import pandas as pd

In [ ]:
def filter_ridges(ridge, 
                  threshold_ridges=0.1, 
                  ridge_opening_element=disk(3)):
    
    # Threshold the ridge image to only retain values higher than a threshold, and to binarize the image
    binary_ridge = ridge.copy()
    binary_ridge[binary_ridge > threshold_ridges] = 1
    binary_ridge[binary_ridge < 1] = 0

    # Morphological opening and removing of small objects
    opened_ridge = opening(binary_ridge, ridge_opening_element)
    opened_ridge[~remove_small_objects(opened_ridge > 0, min_size=128)] = 0
    
    return binary_ridge, opened_ridge

In [ ]:
def extract_roi(img):
    # Downscale the image
    img_small = rescale(img, 0.25, anti_aliasing=False)
    
    # Threshold the image with a fixed threshold
    img_smooth = img_small.copy()
    
    # Smooth the image with a Gaussian filter
    smooth = gaussian(img_smooth, sigma=8)
    smooth = gaussian(smooth, sigma=10)
    
    # Threshold with an Otsu threshold
    thresh_value = threshold_otsu(smooth)
    img_thresh_otsu = smooth > thresh_value

    # Filling holes and convex hull
    img_fill = ndi.binary_fill_holes(img_thresh_otsu)
    img_fill1 = convex_hull_image(img_fill)
    
    # Create a label image
    label_image = label(img_fill1)
    min_area = 5000

    filtered_labels = np.zeros(label_image.shape)
    
    img_regionprops = regionprops(label_image)

    # Only keep regions with areas that are large enough
    for index, region in enumerate(img_regionprops):
        if region.area >= min_area:
            print(index)
            filtered_labels[label_image == (index + 1)] = 1
    
    largest_area = 0
    largest_area_index = 0
    
    if np.max(filtered_labels) == 0:
        for index, region in enumerate(img_regionprops):
            # take regions with large enough areas
            if region.area >= largest_area:
                print(index)
                largest_area_index = index
                largest_area = region.area
        filtered_labels[label_image == (largest_area_index + 1)] = 1
    
    filtered_labels_large = resize(filtered_labels, img.shape, anti_aliasing=False)
    
    return filtered_labels_large

In [ ]:
# Extract ROI and ridges

img_path = '/path_to_data/'

save_path = os.path.join('output_dir')

img_files: list = glob(os.path.join(img_path, '*.tif'))

ridge_densities = []
img_name_list = []

for img_file in img_files:
    print(img_file)
    img = imread(img_file)

    if len(img.shape) == 3:
        img = img[1, :, :]
    
    # Extract region of interest
    filtered_labels_large = extract_roi(img)
    
    # Calculate ridges
    ridge = meijering(img, sigmas=range(1, 10, 2), black_ridges=False)
    
    # Filter ridges
    binary_ridge, opened_ridge = filter_ridges(ridge,
                                               threshold_ridges=0.25,
                                               ridge_opening_element=disk(2))
    
    # Filter the ridges with the ROI
    filtered_ridge = opened_ridge.copy()
    filtered_ridge[filtered_labels_large == 0] = 0
    
    # Calculate the ridge density in the ROI
    ridge_density = sum(sum(filtered_ridge > 0.0001))/sum(sum(filtered_labels_large))
    
    # For display
    filtered_ridge[filtered_ridge > 0.0001] = 1
    filtered_ridge[filtered_ridge < 1] = 0
    
    filtered_ridge[filtered_labels_large > 0] += 0.3
    
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 16))

    ax[0, 0].imshow(ridge, cmap='gray')
    ax[0, 0].set_title(r'Meijering ridges', fontsize=20)
    ax[0, 1].imshow(binary_ridge, cmap='gray')
    ax[0, 1].set_title(r'Thresholded ridges', fontsize=20)
    ax[1, 0].imshow(opened_ridge, cmap='gray')
    ax[1, 0].set_title(r'Opened ridges', fontsize=20)
    ax[1, 1].imshow(filtered_ridge, cmap='gray')
    ax[1, 1].set_title(r'Filtered ridges on filtered ROI', fontsize=20)
    
    for a in ax:
        for a_2 in a:
            a_2.set_axis_off()
    
    img_identifier = str(os.path.basename(img_file).split('.tif')[0])
    plt.savefig(os.path.join(save_path, 'ridges_and_roi_steps_' + img_identifier))
    
    imsave(os.path.join(save_path, 'ridges_and_roi_' + img_identifier + '.png'), filtered_ridge)
    
    ridge_densities.append(ridge_density)
    img_name_list.append(img_identifier)
    
density_dict = {'image': img_name_list,
                'Density': ridge_densities}  
       
df = pd.DataFrame(density_dict) 
    
# Saving the dataframe 
df.to_csv(os.path.join(save_path, 'densities.csv'))